# These are a series of notebooks presented at the 2018 User Conference
They highlight configuring adding users, assignment types, integrations, and assignments using various ways. There are 6 notebooks in the series (exluding this one). They should be followed in order from 1 to 6. By substituting the credentials, layers, and project, with your own information these workflows should be easily replicated.

These notebooks require the ArcGIS API for Python version 1.4.1 or higher as well as shapely or arcpy installed.

In the blocks below, there is some code that can be used to throughout the notebooks (if necessary).

### Build SQLite Database and Table
For notebook number 4, assignments are read from a SQLite database. The following block creates the database using sqlite3 and pandas.

In [ ]:
import sqlite3
from datetime import datetime
import pandas as pd

df = pd.DataFrame(
[
    [1, datetime(2018, 7, 12), "Colombia St & Broadway, San Diego, CA", 1, "Sidewalk Repair", "Completed", "The sidewalk needs to be fixed.", "Done"],
    [2, datetime(2018, 7, 13), "1800 Fifth Ave, San Diego, CA", 1, "Sidewalk Repair", "Completed", "The sidewalk is uneven due to tree roots.", "Finished"],
    [3, datetime(2018, 7, 14), "2115 Imperial Ave, San Diego, CA", 2, "Sidewalk Repair", "Backlog", "The sidewalk is very uneven.", None],
    [4, datetime(2018, 7, 15), "South Evans St & Franklin Ave, San Diego, CA", 2, "Sidewalk Repair", "Backlog", "Please fix the sidewalk near the intersection", None],
    [5, datetime(2018, 7, 16), "Market St & 31st St, San Diego, CA", 3, "Sidewalk Repair", "Backlog", "Fix my side walk", None],
    [6, datetime(2018, 7, 12), "Ivy St & Fern St, San Diego, CA", 3, "Sidewalk Repair", "Backlog", "Fix the side walk in front of my shop", None],
],
columns=["id", "due_date", "address", "priority", "type", "status", "description", "notes"])

connection = sqlite3.connect("work_orders")
df.to_sql("work_orders", connection, if_exists="replace")
df

### Reset Project
The following block can be used to reset the Workforce Project to the original state.

In [ ]:
from arcgis.gis import GIS
from arcgis.apps import workforce
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("1f7b42024da544f6b1e557889e858ac6")
project = workforce.Project(item)

project.assignments_item.layers[0].delete_features(where="1=1")
project.workers.batch_delete(project.workers.search())
project.dispatchers.batch_delete(project.dispatchers.search("userId <> 'workforce_scripts'"))
project.assignment_types.batch_delete(project.assignment_types.search())
project.integrations.batch_delete([project.integrations.get("default-explorer"), project.integrations.get("waze-navigation")])